In [1]:
import pandas as pd
import os

In [2]:
import sklearn as sk
from sklearn.linear_model import MultiTaskLasso, MultiTaskElasticNet, ElasticNet

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import sklearn.metrics as skmet

In [5]:
import numpy as np

In [6]:
os.listdir("inside_airbnb") 

['neighbourhoods.geo.json',
 'reviews.csv',
 '.DS_Store',
 'neighbourhoods.geojson',
 'listings_summ.csv',
 'neighbourhoods.csv',
 'stations.geojson',
 'listings.csv',
 'calendar.csv',
 'reviews_summ.csv',
 'neighbourhoods.json']

In [7]:
listings = pd.read_csv('inside_airbnb/listings.csv')
list_summ = pd.read_csv('inside_airbnb/listings_summ.csv')
neighborhoods = pd.read_csv('inside_airbnb/neighbourhoods.csv')
reviews = pd.read_csv('inside_airbnb/reviews.csv')
reviews_summ = pd.read_csv('inside_airbnb/reviews_summ.csv')
calendar = pd.read_csv('inside_airbnb/calendar.csv')

In [8]:
listings.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

In [9]:
listings = listings.loc[(listings.zipcode==92101) & (listings.room_type == 'Entire home/apt')]

In [10]:
calendar['price'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

In [11]:
calendar['price'] = pd.to_numeric(calendar['price'], errors='coerce')

In [12]:
calendar['price'] = calendar['price']/100

In [13]:
def group_mean(x):
    x['mean_price'] = x['price'].mean()
    return x

In [14]:
fr = calendar.groupby(calendar.listing_id).apply(group_mean)

In [15]:
fr = fr.loc[fr.available!='t']

In [16]:
frame = fr[['listing_id','mean_price']].groupby(['listing_id']).sum()

In [17]:
merged = listings.merge(frame, how='inner', left_on='id', right_index=True)

In [18]:
merged.rename(columns={'mean_price':'estimated_tot_income'}, inplace=True)

In [19]:
merged['estimated_tot_income'] = merged['estimated_tot_income'].apply(lambda x: x*0.75)

In [20]:
cols = ['id','bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet','review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication',
       'review_scores_location', 'review_scores_value', 'estimated_tot_income']

In [21]:
merged[cols].sort_values(['estimated_tot_income'], ascending=False).head(3)

,id,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,estimated_tot_income
5731,6869868,3.0,2.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Air Conditioning"",Kit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,599062.50000
1285,6275771,2.0,1.0,2.0,Real Bed,"{TV,Internet,""Wireless Internet"",""Air Conditio...",NaN,10.0,10.0,10.0,10.0,10.0,10.0,351000.00000
5723,13298618,3.0,3.0,5.0,Real Bed,"{TV,""Wireless Internet"",""Air Conditioning"",Kit...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,341978.04878


In [22]:
calendar.loc[(calendar.listing_id == 6869868) & (calendar.available!='t')].count()

listing_id    355
date          355
available     355
price           0
dtype: int64

In [23]:
#The neighborhoods in San Diego that AirBNB works with
listings['neighbourhood'].unique()

array(['Cortez', 'Core-Columbia', 'Marina', 'Little Italy', 'Harborview',
       nan, 'East Village', 'Park West', 'Gaslamp', 'Petco Park',
       'Horton Plaza', 'Sherman Heights', 'Midway District', 'Midtown'], dtype=object)

In [24]:
list_summ['neighbourhood'].unique()

array(['Amphitheater And Water Park', 'East Lake', 'Eastlake Trails',
       'Eastlake Vistas', 'Estlake Greens', 'Lynwood Hills', 'Otay Ranch',
       'Northwest', 'Yosemite Dr', 'Allied Gardens', 'Paseo Ranchoero',
       'Rancho Del Rey', 'Rolling Hills Ranch', 'Alta Vista',
       'Balboa Park', 'Southwest', 'Sunbow', 'Terra Nova',
       'Thomy Locust Pl', 'Bario Logan', 'Bay Ho', 'Bay Park',
       'Bay Terrace', 'Bird Land', 'Carmel Mountain', 'Carmel Valley',
       'Chollas View', 'City Heights East', 'City Heights West',
       'Clairemont Mesa', 'College Area', 'Columbia', 'Core',
       'Cortez Hill', 'Darnall', 'Del Cerro', 'Marina', 'Del Mar Heights',
       'East Village', 'Egger Highlands', 'El Cerritos', 'Memorial',
       'Encanto', 'Gaslamp Quarter', 'Gateway', 'Grant Hill', 'Grantville',
       'Horton Plaza', 'Jomacha-Lomita', 'Kearny Mesa', 'Kensington',
       'Midtown', 'Moreno Mission', 'La Jolla', 'La Jolla Village',
       'Lake Murray', 'Lincoln Park', 'Lind

In [25]:
L = []
for w in list_summ['neighbourhood'].unique():
    L.append((w, list_summ[list_summ['neighbourhood'] == w]['price'].mean()))
    
L = sorted(L, key = lambda x: x[1], reverse = True)

for w in L:
    print w

('La Jolla', 478.50506756756755)
('Wooded Area', 454.56)
('North City', 393.95999999999998)
('Columbia', 311.3125)
('Torrey Pines', 308.23076923076923)
('Del Mar Heights', 300.77464788732397)
('Mission Bay', 287.51837769328262)
('Midtown District', 287.24489795918367)
('Rancho Del Rey', 267.57142857142856)
('Little Italy', 249.70526315789473)
('Rancho Bernadino', 241.02500000000001)
('San Ysidro', 240.83333333333334)
('San Carlos', 238.20689655172413)
('Horton Plaza', 235.0)
('Gaslamp Quarter', 234.0)
('Carmel Valley', 221.86567164179104)
('Roseville', 221.31147540983608)
('Ocean Beach', 210.64102564102564)
('Cortez Hill', 208.89743589743588)
('Core', 206.1081081081081)
('Del Cerro', 198.72727272727272)
('East Village', 198.34545454545454)
('Rancho Penasquitos', 196.09090909090909)
('Kensington', 194.09090909090909)
('Old Town', 193.28099173553719)
('Park West', 192.15686274509804)
('Pacific Beach', 192.01536098310291)
('Lynwood Hills', 188.0)
('Midtown', 173.12025316455697)
('Grant Hi

In [26]:
listings['amenities'].head(4)

226    {TV,"Cable TV",Internet,"Wireless Internet","A...
781    {TV,"Cable TV",Internet,"Wireless Internet","A...
783    {"Air Conditioning","Family/Kid Friendly",Wash...
784    {TV,"Cable TV",Internet,"Wireless Internet","A...
Name: amenities, dtype: object

In [27]:
list_summ.columns

Index([u'id', u'name', u'host_id', u'host_name', u'neighbourhood_group',
       u'neighbourhood', u'latitude', u'longitude', u'room_type', u'price',
       u'minimum_nights', u'number_of_reviews', u'last_review',
       u'reviews_per_month', u'calculated_host_listings_count',
       u'availability_365'],
      dtype='object')

In [28]:
listings['has_availability'].unique()

array([ nan])

In [29]:
listings['availability_30'].head(3)

226    0
781    0
783    0
Name: availability_30, dtype: int64

In [30]:
sorted(listings['availability_30'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30]

In [31]:
sorted(listings['availability_60'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60]

In [32]:
sorted(listings['availability_90'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90]

In [33]:
sorted(listings['availability_365'].unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 25,
 27,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 50,
 51,
 52,
 53,
 55,
 56,
 57,
 58,
 63,
 66,
 67,
 68,
 69,
 71,
 72,
 73,
 74,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 96,
 99,
 101,
 103,
 105,
 106,
 112,
 116,
 117,
 120,
 122,
 123,
 128,
 131,
 133,
 135,
 137,
 138,
 140,
 141,
 144,
 145,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 156,
 157,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 167,
 173,
 178,
 179,
 186,
 190,
 191,
 192,
 199,
 200,
 204,
 206,
 208,
 211,
 212,
 215,
 219,
 220,
 221,
 224,
 225,
 226,
 227,
 228,
 229,
 232,
 233,
 234,
 236,
 237,
 250,
 251,
 253,
 259,
 260,
 263,
 264,
 265,
 266,
 268,
 270,
 271,
 274,
 275,
 276,
 277,
 279,
 280,
 281,
 284,
 285,
 288,
 290,
 292,
 293,
 294,
 296,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 

In [34]:
calendar.head(3)

,listing_id,date,available,price
0,11204286,2017-07-05,t,49.0
1,11204286,2017-07-04,t,49.0
2,11204286,2017-07-03,t,49.0


In [35]:
calendar['available'].unique()

array(['t', 'f'], dtype=object)

In [36]:
reviews.head(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,4796400,32687022,2015-05-20,29916246,Bill,"Jeff, we cannot thank you enough for your hosp..."
1,4796400,33062004,2015-05-25,3547281,Christina,"The location was amazing, we were able to chea..."
2,4796400,41311067,2015-08-07,16458913,Naif,I would like to thank Jeff for his great hospi...


In [37]:
reviews_summ.head(3)

,listing_id,date
0,4796400,2015-05-20
1,4796400,2015-05-25
2,4796400,2015-08-07


In [38]:
neighborhoods.head(3)

,neighbourhood_group,neighbourhood
0,NaN,Allied Gardens
1,NaN,Alta Vista
2,NaN,Amphitheater And Water Park


In [39]:
calendar['price'].unique()

array([   49.,    nan,    59., ...,   991.,  2240.,  1854.])

In [40]:
merged.head(3)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month,estimated_tot_income
226,6902290,https://www.airbnb.com/rooms/6902290,20160706203047,2016-07-07,Comic Con! Two Bedroom Condo,"2-Bedroom Condo in Cortez Hill, downtown San D...",NaN,"2-Bedroom Condo in Cortez Hill, downtown San D...",none,NaN,...,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",f,strict,f,f,2,0.08,NaN
781,6899514,https://www.airbnb.com/rooms/6899514,20160706203047,2016-07-07,Spectacular Views Little Italy,Walk to the San Diego night life. Awesome rest...,Decorated with high end furniture; spectacular...,Walk to the San Diego night life. Awesome rest...,none,Awesome restaurants; spectacular views; alive ...,...,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",t,strict,f,f,1,0.09,NaN
783,8407570,https://www.airbnb.com/rooms/8407570,20160706203047,2016-07-07,clean and neat,great!,NaN,great!,none,NaN,...,f,NaN,"SAN DIEGO, SAN DIEGO TOURISM MARKETING DISTRIC...",f,flexible,f,f,1,NaN,NaN


In [41]:
merged['price'].head(3)

226    $600.00
781    $350.00
783    $999.00
Name: price, dtype: object

In [42]:
calendar['price'].head(3)

0    49.0
1    49.0
2    49.0
Name: price, dtype: float64

In [43]:
L = []

for w in merged.index:
    a = 0
    b = merged['price'][w][1:].split(',')
    c = ''
    
    for k in b:
        c = c + k
    
    L.append(int(float(c)))

In [44]:
merged['numerical price'] = L

In [45]:
merged['numerical price'].head(3)

226    600
781    350
783    999
Name: numerical price, dtype: int64

In [46]:
merged.columns

Index([u'id', u'listing_url', u'scrape_id', u'last_scraped', u'name',
       u'summary', u'space', u'description', u'experiences_offered',
       u'neighborhood_overview', u'notes', u'transit', u'access',
       u'interaction', u'house_rules', u'thumbnail_url', u'medium_url',
       u'picture_url', u'xl_picture_url', u'host_id', u'host_url',
       u'host_name', u'host_since', u'host_location', u'host_about',
       u'host_response_time', u'host_response_rate', u'host_acceptance_rate',
       u'host_is_superhost', u'host_thumbnail_url', u'host_picture_url',
       u'host_neighbourhood', u'host_listings_count',
       u'host_total_listings_count', u'host_verifications',
       u'host_has_profile_pic', u'host_identity_verified', u'street',
       u'neighbourhood', u'neighbourhood_cleansed',
       u'neighbourhood_group_cleansed', u'city', u'state', u'zipcode',
       u'market', u'smart_location', u'country_code', u'country', u'latitude',
       u'longitude', u'is_location_exact', u'prope

In [47]:
merged['bathrooms'].head(3)

226    2.0
781    2.0
783    1.0
Name: bathrooms, dtype: float64

In [48]:
merged['bathrooms'].unique()

array([ 2. ,  1. ,  1.5,  2.5,  3. ,  nan,  5. ])

In [49]:
merged['house_rules'].head(3)

226                                                  NaN
781    no drugs or excessive alcohol. Respect of the ...
783                                                  NaN
Name: house_rules, dtype: object

In [50]:
def is_transit(a):
    if a.len() > 0:
        return 1
    else:
        return 0

In [49]:
merged['transit'].head(3)

226    NaN
781    NaN
783    NaN
Name: transit, dtype: object

In [52]:
L = []
for w in merged.index:
    a = merged['transit'].str.len()[w]
    if  a > 0:
        L.append(1)
    else:
        L.append(0)

merged['is_transit'] = L

In [53]:
merged['is_transit'].head(3)

226    0
781    0
783    0
Name: is_transit, dtype: int64

In [54]:
merged['transit'].str.split(' ')[788]

['One',
 'Free',
 'parking',
 'space',
 'is',
 'available',
 'if',
 'driving',
 'and',
 'access',
 'to',
 'the',
 'trolley',
 'is',
 'minutes',
 'away.',
 'There',
 'are',
 'also',
 'buses',
 'and',
 'ride',
 'shares',
 'easily',
 'accessible',
 'from',
 'this',
 'location.']

In [55]:
merged['neighbourhood_cleansed'].head(3)

226    Balboa Park
781       Columbia
783       Columbia
Name: neighbourhood_cleansed, dtype: object

In [56]:
merged['neighbourhood'].head(3)

226           Cortez
781    Core-Columbia
783    Core-Columbia
Name: neighbourhood, dtype: object

In [57]:
reviews['comments'].head(3)

0    Jeff, we cannot thank you enough for your hosp...
1    The location was amazing, we were able to chea...
2    I would like to thank Jeff for his great hospi...
Name: comments, dtype: object

In [58]:
reviews['comment_length'] = reviews['comments'].str.len()

# Modeling

## Model 1

In [59]:
clf = ElasticNet()

In [60]:
merged_drop_NA = merged[['bathrooms', 'bedrooms', 'numerical price']].dropna()

In [61]:
X_train, X_test, y_train, y_test = train_test_split(merged_drop_NA[['bathrooms', 'bedrooms']], merged_drop_NA['numerical price'], test_size=0.2)

In [62]:
clf.fit(X_train, y_train)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [63]:
y_pred = clf.predict(X_test)

In [64]:
y_pred

array([ 233.16971662,  302.02623154,  197.39404242,  266.25055734,
        302.02623154,  268.94539082,  233.16971662,  233.16971662,
        285.48581118,  266.25055734,  302.02623154,  233.16971662,
        197.39404242,  233.16971662,  197.39404242,  233.16971662,
        233.16971662,  233.16971662,  302.02623154,  197.39404242,
        249.71013698,  233.16971662,  266.25055734,  266.25055734,
        442.43409486,  302.02623154,  197.39404242,  354.3423261 ,
        233.16971662,  197.39404242,  197.39404242,  233.16971662,
        233.16971662,  197.39404242,  197.39404242,  197.39404242,
        233.16971662,  233.16971662,  302.02623154,  302.02623154,
        268.94539082,  233.16971662,  233.16971662,  197.39404242,
        233.16971662,  233.16971662,  233.16971662,  233.16971662,
        197.39404242,  266.25055734,  233.16971662,  268.94539082,
        233.16971662,  233.16971662,  197.39404242,  268.94539082,
        197.39404242,  233.16971662,  233.16971662,  268.94539

In [65]:
print('The Mean Squared Error of the first model = ' + str(mean_squared_error(y_pred, y_test)))
print('The Mean Absolute Error of the first model = ' + str(mean_absolute_error(y_pred, y_test)))

The Mean Squared Error of the first model = 22988.0156788
The Mean Absolute Error of the first model = 109.388063582


In [56]:
merged['house_rules']

226                                                   NaN
781     no drugs or excessive alcohol. Respect of the ...
783                                                   NaN
784                                                   NaN
785                                                   NaN
786                                    No smoking please!
787     This is our place. We want it to feel like it'...
788            No smoking, no loud music, no pets allowed
789     Respectful, Quiet past 10pm, no ragers/sex par...
790                                                   NaN
791                          Small dogs are OK.  No cats.
792     Personal items in the house, just be respectfu...
793     Please treat my place as it's yours and keep i...
794     RULES AND REGULATIONS FOR artist flat The part...
795                                                   NaN
796     House Rules: • Please no shoes in the house as...
798                                                   NaN
800     Please

In [57]:
def is_transit(a):
    if a.len() > 0:
        return 1
    else:
        return 0

In [58]:
merged['transit']

226                                                   NaN
781                                                   NaN
783                                                   NaN
784                                                   NaN
785     Uber, Lyft, rental car, black car services or ...
786                                                   NaN
787                                                   NaN
788     One Free parking space is available if driving...
789     We are in the heart of all of downtown, close ...
790                                                   NaN
791     Stay in the most walkable neighborhood in San ...
792     *Uber*Car2go *Taxi Services *Walk to Buses &Tr...
793     Santa Fe Depot (Amtrack/Coaster) is a block an...
794     A 5 min walk to the Trolly that can take you a...
795     The Urban Treehouse is conveniently located in...
796     **Free Parking in secured garage!!** Conventio...
798                                                   NaN
800           

In [59]:
L = []
for w in merged.index:
    a = merged['transit'].str.len()[w]
    if  a > 0:
        L.append(1)
    else:
        L.append(0)

merged['is_transit'] = L

In [67]:
merged['is_transit']

226     0
781     0
783     0
784     0
785     1
786     0
787     0
788     1
789     1
790     0
791     1
792     1
793     1
794     1
795     1
796     1
798     0
800     0
802     0
803     0
804     0
805     1
806     1
807     1
809     0
812     1
813     1
814     1
819     1
820     1
       ..
5738    0
5739    1
5740    1
5742    1
5743    1
5744    1
5745    1
5746    0
5748    1
5750    1
5754    1
5755    0
5756    1
5757    1
5758    0
5759    1
5760    1
5761    1
5762    1
5764    1
5766    0
5767    1
5768    1
5769    0
5770    1
5773    1
5775    1
5778    0
5781    1
5782    1
Name: is_transit, Length: 581, dtype: int64

In [68]:
merged['transit'].str.split(' ')[788]

['One',
 'Free',
 'parking',
 'space',
 'is',
 'available',
 'if',
 'driving',
 'and',
 'access',
 'to',
 'the',
 'trolley',
 'is',
 'minutes',
 'away.',
 'There',
 'are',
 'also',
 'buses',
 'and',
 'ride',
 'shares',
 'easily',
 'accessible',
 'from',
 'this',
 'location.']

In [69]:
merged['neighbourhood_cleansed']

226     Balboa Park
781        Columbia
783        Columbia
784        Columbia
785        Columbia
786        Columbia
787        Columbia
788        Columbia
789        Columbia
790        Columbia
791        Columbia
792        Columbia
793        Columbia
794        Columbia
795        Columbia
796        Columbia
798            Core
800            Core
802            Core
803            Core
804            Core
805            Core
806            Core
807            Core
809            Core
812            Core
813            Core
814            Core
819            Core
820            Core
           ...     
5738      Park West
5739      Park West
5740      Park West
5742      Park West
5743      Park West
5744      Park West
5745      Park West
5746      Park West
5748      Park West
5750      Park West
5754      Park West
5755      Park West
5756      Park West
5757      Park West
5758      Park West
5759      Park West
5760      Park West
5761      Park West
5762      Park West


In [70]:
merged['neighbourhood']

226            Cortez
781     Core-Columbia
783     Core-Columbia
784            Marina
785      Little Italy
786        Harborview
787               NaN
788               NaN
789     Core-Columbia
790     Core-Columbia
791     Core-Columbia
792      Little Italy
793     Core-Columbia
794               NaN
795      Little Italy
796     Core-Columbia
798     Core-Columbia
800     Core-Columbia
802      East Village
803     Core-Columbia
804      East Village
805     Core-Columbia
806      East Village
807               NaN
809      East Village
812     Core-Columbia
813               NaN
814               NaN
819      East Village
820     Core-Columbia
            ...      
5738        Park West
5739        Park West
5740        Park West
5742        Park West
5743        Park West
5744        Park West
5745        Park West
5746              NaN
5748        Park West
5750              NaN
5754        Park West
5755        Park West
5756        Park West
5757        Park West
5758      

In [77]:
reviews['comments']

0        Jeff, we cannot thank you enough for your hosp...
1        The location was amazing, we were able to chea...
2        I would like to thank Jeff for his great hospi...
3        Jeff was an amazing host! We rented all three ...
4        Jeff was the consummate host.  He was quick to...
5             The location was excellent, literally ste...
6        Wonderful beach house. So close to the beach. ...
7        perfect place for me and my family. would love...
8        Ryan and his staff were amazing!!!!! So incred...
9        My communication with Ryan before our stay was...
10       The location could not have been better.  This...
11       Very nice family who makes you feel welcome an...
12       Clean, friendly, comfortable! Maisue and her f...
13       Maisue was a great host to me and my friend th...
14       Maisue and her family were wonderful hosts. We...
15       Mai Sue and Thé are excellent hosts. I stayed ...
16       Me and my sister stood at Maisue.  We loved th.

In [75]:
reviews['comment_length'] = reviews['comments'].str.len()

# Modeling

## Model 1

In [60]:
clf = ElasticNet()

In [61]:
merged_drop_NA = merged[['bathrooms', 'bedrooms', 'numerical price']].dropna()

In [62]:
X_train, X_test, y_train, y_test = train_test_split(merged_drop_NA[['bathrooms', 'bedrooms']], merged_drop_NA['numerical price'], test_size=0.2)

In [63]:
clf.fit(X_train, y_train)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [64]:
y_pred = clf.predict(X_test)

In [65]:
y_pred

array([ 362.98161807,  193.22505885,  228.82090712,  193.22505885,
        193.22505885,  260.30541432,  264.4167554 ,  300.01260368,
        228.82090712,  193.22505885,  228.82090712,  228.82090712,
        228.82090712,  260.30541432,  193.22505885,  228.82090712,
        228.82090712,  228.82090712,  228.82090712,  295.9012626 ,
        228.82090712,  228.82090712,  228.82090712,  193.22505885,
        295.9012626 ,  228.82090712,  193.22505885,  264.4167554 ,
        331.49711088,  193.22505885,  193.22505885,  228.82090712,
        228.82090712,  228.82090712,  295.9012626 ,  228.82090712,
        362.98161807,  228.82090712,  280.159009  ,  193.22505885,
        228.82090712,  295.9012626 ,  228.82090712,  193.22505885,
        228.82090712,  193.22505885,  228.82090712,  193.22505885,
        331.49711088,  295.9012626 ,  264.4167554 ,  295.9012626 ,
        193.22505885,  228.82090712,  193.22505885,  228.82090712,
        193.22505885,  193.22505885,  228.82090712,  280.15900

In [66]:
print('The Mean Squared Error of the first model = ' + str(mean_squared_error(y_pred, y_test)))
print('The Mean Absolute Error of the first model = ' + str(mean_absolute_error(y_pred, y_test)))

The Mean Squared Error of the first model = 31261.2586994
The Mean Absolute Error of the first model = 119.71305226
